In [ ]:
import pandas as pd 
import sqlite3 

In [ ]:
conn = sqlite3.connect('/kaggle/input/chinook-sqlite/Chinook_Sqlite.sqlite')
c = conn.cursor()

In [ ]:
def sq(str,con=conn):
    return pd.read_sql('''{}'''.format(str), con)

#### 1. Write a query that returns each genre, with the number of tracks sold in the USA:
*    in absolute numbers
*    in percentages

In [ ]:
sq('''SELECT g.name,
        c.Country,
        COUNT(il.InvoiceLineId) AS NumOfTracksSold,
        (CAST(COUNT(il.InvoiceLineId) AS FLOAT)/
        (SELECT (COUNT(i.InvoiceId))
         FROM Invoice
         INNER JOIN
         InvoiceLine il ON t.TrackId = il.TrackId
         INNER JOIN
         Invoice i ON i.InvoiceId = il.InvoiceId
         WHERE c.Country = "USA")) AS PercentOfTracksSold

FROM Genre g
       INNER JOIN
       Track t ON g.GenreId = t.GenreId
       INNER JOIN
       InvoiceLine il ON t.TrackId = il.TrackId
       INNER JOIN
       Invoice i ON i.InvoiceId = il.InvoiceId
       INNER JOIN
       Customer c ON c.CustomerId = i.CustomerId

WHERE 
    c.Country = "USA"
GROUP BY 1
ORDER BY NumOfTracksSold DESC
LIMIT 10;
''')

#### 1. Write a query that finds the total dollar amount of sales assigned to each sales support agent within the company. Add any extra attributes for that employee that you find are relevant to the analysis.

In [ ]:
sq('''WITH Support_Agent AS
(SELECT
    EmployeeId AS SupportAgentId,
    (e.FirstName || " " || e.LastName) AS Name,
    SUM(i.Total) AS Total
    
 FROM
     Employee e
    JOIN Customer c ON c.SupportRepId = e.EmployeeId
    JOIN Invoice i ON i.CustomerId = c.CustomerId
    
 WHERE
     EmployeeId IN (3, 4, 5)
 GROUP BY 
     Name, c.CustomerId
)

SELECT
    sa.Name,
    SUM(i.Total) AS Total

FROM 
    Support_Agent sa
    JOIN Customer c ON c.CustomerId = sa.SupportAgentId
    JOIN Invoice i ON i.CustomerId = c.CustomerId
        
GROUP BY Name
''')

#### 1. Write a query that collates data on purchases from different countries.
   *  Where a country has only one customer, collect them into an "Other" group.
   *  The results should be sorted by the total sales from highest to lowest, with the "Other" group at the very bottom.
   
#### 2. For each country, include:
   *  total number of customers
   *  total value of sales
   *  average value of sales per customer
   *  average order value

In [ ]:
sq('''WITH OtherCountry AS
(SELECT
    CASE
        WHEN (SELECT  COUNT(country)
                 FROM Customer
                 WHERE Country = c.Country) = 1  THEN 'Other'
         ELSE c.Country
     END AS Country, 
     c.CustomerId,
     il.*

 FROM Customer c
 JOIN Invoice i ON c.CustomerId = i.CustomerId
 JOIN InvoiceLine il ON i.InvoiceId = il.InvoiceId
)

SELECT
    Country,
    TotalCustomer,
    TotalSales,
    AvgSalesPerCustomer,
    AvgOrderValue

FROM
    (SELECT 
        Country,
        COUNT(CustomerId) AS TotalCustomer,
        CASE
            WHEN Country = 'Other' THEN 1
            ELSE 0
        END AS sort,
        ROUND(SUM(UnitPrice), 2) AS TotalSales,
        ROUND(SUM(UnitPrice)/COUNT(DISTINCT CustomerId), 2) AS AvgSalesPerCustomer,
        ROUND(SUM(UnitPrice)/Count(InvoiceLineId), 2) AS AvgOrderValue
     
     FROM OtherCountry
     
     GROUP BY Country
     ORDER BY sort, TotalSales DESC
    )
''')